In [2]:
import pandas as pd
import plotly.graph_objects as go

# Load data
file_path = "../logs/512G/2025-01-04-21-51-50-mainnet-beta-memory.log"

# Skip initial lines and load data
data = []
with open(file_path, 'r') as file:
    for line in file:
        if "Timestamp" in line or line.strip().startswith("202"):
            data.append(line.strip().split(','))

# Convert to DataFrame
columns = ['Timestamp', 'PID', '%MEM', 'VIRT', 'RES']
df = pd.DataFrame(data[1:], columns=columns)  # Skip header row
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Convert memory columns to numeric (GB)
def parse_memory(mem):
    if mem[-1] == 'g':  # Gigabytes
        return float(mem[:-1])
    elif mem[-1] == 'm':  # Megabytes
        return float(mem[:-1]) / 1024
    else:  # Assume KB
        return float(mem) / (1024 ** 2)

df['VIRT (GB)'] = df['VIRT'].apply(parse_memory)
df['RES (GB)'] = df['RES'].apply(parse_memory)
df['%MEM'] = df['%MEM'].astype(float)

# Calculate elapsed time since start
start_time = df['Timestamp'].min()
df['Elapsed Time (seconds)'] = (df['Timestamp'] - start_time).dt.total_seconds()  # Total seconds elapsed
df['Elapsed Time (hours)'] = df['Elapsed Time (seconds)'] / 3600  # Convert to hours

# Plot using Plotly
fig = go.Figure()

# Add traces for VIRT and RES
fig.add_trace(go.Scatter(
    x=df['Elapsed Time (hours)'],
    y=df['VIRT (GB)'],
    mode='lines',
    name='VIRT (GB)',
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=df['Elapsed Time (hours)'],
    y=df['RES (GB)'],
    mode='lines',
    name='RES (GB)',
    line=dict(color='orange')
))

# Update layout
fig.update_layout(
    title="Memory Usage Over Time",
    xaxis_title="Time Elapsed (hours)",
    yaxis_title="Memory (GB)",
    legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0)'),
    template="plotly_white"
)

# Show plot
fig.show()

# Highlight statistics
max_res = df['RES (GB)'].max()
max_virt = df['VIRT (GB)'].max()
avg_res = df['RES (GB)'].mean()
avg_virt = df['VIRT (GB)'].mean()

print(f"Max RES (GB): {max_res:.2f}")
print(f"Max VIRT (GB): {max_virt:.2f}")
print(f"Average RES (GB): {avg_res:.2f}")
print(f"Average VIRT (GB): {avg_virt:.2f}")

Max RES (GB): 497.70
Max VIRT (GB): 1044.70
Average RES (GB): 379.95
Average VIRT (GB): 823.31
